In [1]:
#Implementation of A*
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2 as cv
from tqdm import tqdm
from tensorflow.keras.models import load_model
import os
from dataclasses import dataclass
import random

global X


In [2]:
#All combs


aux = [
            [0,1,2],
            [0,1,2,3],
            [0,1,2],
            [1,2,3],
            [1,2],
            [1,2,3,4],
            [1,2,3,4],
            [1,2,3,4],
            [0,1,2]

        ]


                            

In [3]:
@dataclass
class node:
    f : float 
    g : float
    h : float
    comb : list
    parent : list
    prob : float
        

In [11]:
#Definir funções

media_dos_valores = 0.83

discriminator = load_model(os.getcwd() + "\discriminator_model8100.h5")






def calculate_prob(comb):
        
    X_train, chars = [np.asarray([X]), np.asarray([comb])]

    next_prob = discriminator.predict([X_train, chars])[0][0]
    
    return next_prob
    

# G -> Custo de um movimento de uma combinação para outra

def g(current,next_comb):
    


   
    #Current prob
    X_train, chars = [np.asarray([X]), np.asarray([current])]

    current_prob = discriminator.predict([X_train, chars])[0][0]
    


    #Next prob
    
    
    X_train, chars = [np.asarray([X]), np.asarray([next_comb])]

    next_prob = discriminator.predict([X_train, chars])[0][0]
    
    return current_prob - next_prob



    




# H -> A estimação do atual para a final 
def h(comb):
    
    

    X_train, chars = [np.asarray([X]), np.asarray([comb])]

    actual_probability = discriminator.predict([X_train, chars])[0][0]
    
    return media_dos_valores - actual_probability



# F -> Sum of the two above


def f(h_value,g_value):
    return h_value + g_value

In [5]:
def get_sucessors(comb,fixed):
    
    sucessors = []
    
    for el in range(len(aux)):
        
        
        if fixed[el] == 0:
        
            secondary = comb.copy()


            if comb[el] + 1 in aux[el]:

                secondary[el] = secondary[el] + 1
                sucessors.append(secondary)



            secondary = comb.copy()


            if comb[el] - 1 in aux[el]:

                secondary[el] = secondary[el] - 1

                sucessors.append(secondary)



        
    return sucessors
        

     
                
                

In [6]:
def get_random_combination_start():
    
    
    
    combination_random = []
    for i in aux:
        combination_random.append(random.choice(i))
    
    return combination_random

In [7]:
    
def create_node(comb,parent,h_v,g_v,f_v,x):
   

    return node(f_v,g_v,h_v,comb,parent,x)
    


In [12]:

aux2 = pickle.load(open(os.getcwd() + "//needed_correct_images.p", "rb"))
ks = list(aux2.keys())


probs_for_each_image = []


ks = ks[8000:]


# Find the image a given directory
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

path_to_watch = os.getcwd() + "\\annotations\\"


for image in tqdm(ks):
    
    size = (192, 256)

    im = cv.imread(find(image,os.getcwd() + "//annotations//"))
    im = cv.cvtColor(im, cv.COLOR_BGR2RGB)

    normalized_image = np.zeros(size)
    normalized_image = cv.normalize(im, normalized_image, 0, 255, cv.NORM_MINMAX)
    resize = cv.resize(normalized_image, size, interpolation=cv.INTER_AREA)

    X = (np.asarray(resize) - 127.5) / 127.5
    
    
    
    actual_combination = aux2[image]

    X_train, chars = [np.asarray([X]), np.asarray([actual_combination])]

    actual_probability = discriminator.predict([X_train, chars])[0][0]
    
    
    
    while(True):
        #we initializae the open and close the list
        open_list = []
        closed_list = []

        start_comb = get_random_combination_start()



        open_list.append(create_node(start_comb,[],h(start_comb),0,0,0))
        counter = 0
        winner = None
        to_stay = [0,0,0,0,0,0,0,0,0]



        while(True):

            maxim = 999
            q = None
            auxiliar = 0
            guardador = 0

            finished = False
            control = False
            control2 = False


            #Find lowest f in open_list
            for el in open_list:
                if el.f < maxim:
                    maxim = el.f
                    guardador = auxiliar
                auxiliar +=1


            q = open_list.pop(guardador)


            q_sucessors = get_sucessors(q.comb,to_stay)


            for sucessor in q_sucessors:


                #Caso seja o nodo destino
                print(calculate_prob(sucessor))
                if calculate_prob(sucessor) >= media_dos_valores:
                    finished = True
                    winner = sucessor


                #Calculamos o g , h e f do sucessor
                g_value = q.g + (g(q.comb,sucessor))

                h_value = h(sucessor)


                f_value = f(g_value,h_value)


                actual = create_node(sucessor,q.comb,h_value,g_value,f_value,calculate_prob(sucessor))




                #COrrer open list
                for o in open_list:
                    if sucessor == o.comb:
                        if o.f < actual.f:
                            control = True



                if not control:
                    #COrrer closed list
                    for c in closed_list:
                        if sucessor == c.comb:
                            if c.f < actual.f:
                                control2 = True

                if not control2:
                    open_list.append(actual)



            closed_list.append(q)



            to_stay[counter] = 0


            if((len(open_list) == 0) or finished ):
                break


            counter +=1


            if counter == 9:
                counter = 0
                
        
        min = 0    
        
        to_save = None
        for p in closed_list:
            if p.prob > min :
                min = p.prob
                to_save = p 
        
        
        if to_save.h <0.05:
            
            
            probs_for_each_image.append([actual_combination, actual_probability, to_save.prob, to_save.comb])
            pickle.dump(probs_for_each_image, open("all_probabilities_estrela.p", "wb"))
            break





    
    

    


                                     






  0%|                                                                                         | 0/2250 [00:00<?, ?it/s]

0.2503955
0.22422872
0.22329459
0.22467154
0.25135532
0.22009192
0.22436978
0.22153643
0.22096604
0.23099466
0.24977517
0.21901877
0.29523116
0.22172317
0.2296463
0.70703566
0.22846971
0.23845938
0.22128096
0.23337774
0.22185591
0.21978569
0.23127691
0.25771484
0.21932171
0.2982302
0.3659989
0.36114207
0.5471387
0.25135532
0.22467154
0.2904578
0.26082954
0.7500447
0.5551142
0.2299301
0.23074152
0.7151319
0.31432894
0.35175902
0.44495687
0.68630975
0.36939645
0.23337774
0.35342366
0.44767445
0.31059933
0.8331047
0.70703566
0.22895247
0.21165884
0.22728036
0.81853175
0.2232022
0.21244207
0.3319683
0.23392431
0.2160184
0.28524292
0.21458635
0.27537605
0.22101685
0.24085501
0.22309373
0.21395843
0.21318743
0.21431606
0.28879294
0.21456271
0.23704441
0.35340044
0.21856594
0.21517505
0.22538374
0.7328838
0.22795773
0.60404277
0.26473144
0.21620189
0.21553649
0.21919547
0.23253244
0.21515006
0.24120533
0.23253244
0.72596204
0.48488796
0.42414683
0.36257777
0.5852253
0.22859567
0.68325895
0.25




  0%|                                                                              | 1/2250 [01:17<48:13:48, 77.20s/it]

0.23688982
0.23901309
0.25351968
0.24928673
0.2559227
0.24747954
0.31843024
0.33065608
0.24261597
0.24618489
0.3528257
0.34832206
0.26152745
0.487963
0.42464116
0.34650612
0.2760531
0.47949058
0.56418985
0.2790114
0.2504317
0.69065416
0.30551684
0.7394242
0.7615448
0.7126952
0.5952442
0.56463146
0.3528257
0.47090068
0.32007688
0.33065608
0.38836154
0.30022112
0.8198802
0.62758595
0.56418985
0.8036401
0.8065168
0.7074984
0.42464116
0.41442522
0.7860528
0.6528436
0.68284124
0.50080866
0.7507683
0.7615448
0.7713097
0.7394242
0.8270201
0.80322224
0.8139952
0.7584385
0.70793694
0.8028841
0.7054078
0.8018234
0.5902634
0.8057348
0.8036401
0.83079875
0.8105883
0.51919264
0.8198802
0.80348635
0.8016015
0.564514
0.5054423
0.84029365
0.76791805





  0%|                                                                              | 2/2250 [01:44<38:50:27, 62.20s/it]

...




  0%|                                                                              | 3/2250 [02:09<31:52:34, 51.07s/it]

0.41449317
0.2263987
0.2247678
0.23102571
0.21936563
0.22081082
0.22022921
0.3119666
0.24315192
0.24084681
0.22122164
0.22828318
0.25345585
0.2611829
0.2908363
0.6343968
0.24206083
0.24868068
0.30876175
0.42480138
0.26113245
0.4881434
0.3664581
0.42480138
0.23095487
0.31061748
0.24403155
0.26709458
0.2612257
0.24033245
0.23596933
0.22828318
0.47102126
0.22696401
0.3119666
0.26490524
0.25978747
0.37167177
0.45498857
0.29151812
0.31807616
0.42303157
0.2831605
0.41449317
0.78147984
0.4729158
0.47102126
0.59314805
0.35867023
0.44679573
0.7129229
0.76963943
0.7377047
0.5660974
0.36421037
0.4881434
0.42480138
0.84072787
0.25117654





  0%|▏                                                                             | 4/2250 [02:32<26:33:18, 42.56s/it]

0.28304946
0.23802191
0.22323479
0.23245013
0.22450894
0.2204157
0.23437782
0.22834305
0.24257533
0.22507243
0.22929372
0.24071606
0.21926136
0.24103849
0.22615294
0.24491443


KeyboardInterrupt: 

In [ ]:
calculate_prob([0,0,0,1,1,1,1,1,0])

[1, 1, 1, 3, 2, 1, 3, 2, 0]

In [ ]:
analytics = pickle.load(open(os.getcwd() + "\\all_probabilities.p","rb"))

In [ ]:
print(analytics[15])

In [ ]:

import matplotlib.cm as cm



matrizes = [
    
            np.zeros((3,3)),
            np.zeros((4,4)),
            np.zeros((3,3)),
            np.zeros((3,3)),
            np.zeros((2,2)),
            np.zeros((4,4)),
            np.zeros((4,4)),
            np.zeros((4,4)),
            np.zeros((3,3)),

]

for el in tqdm(analytics):
    
    predict_comb = el[0]
    
    true_comb = el[-1]
    
    
    for i in range(len(predict_comb)):
        
        
        try:
        
        
            aux2 = true_comb[i]

            aux1 = predict_comb[i]
            matrizes[i][aux[i].index(aux1)][aux[i].index(aux2)] = matrizes[i][aux[i].index(aux2)][aux[i].index(aux1)] + 1
            
        except:
           pass


count = 1
for m in range(len(matrizes)):
    plt.matshow(matrizes[m])
    count +=1

    